# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

In [0]:
from google.colab import files
files.upload()

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [0]:
housing_df = pd.read_csv('cal_housing_clean.csv')

In [0]:
housing_df.head()

In [0]:
housing_df['medianHouseValue'] = np.log(housing_df['medianHouseValue'])

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
target = housing_df['medianHouseValue']
X_train, X_test, Y_train, Y_test = train_test_split(housing_df, target, test_size=0.3, random_state=101)

In [115]:
X_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
6761,19.0,2742.0,756.0,1396.0,703.0,2.5663,197500.0
3010,32.0,444.0,102.0,242.0,87.0,1.1528,150000.0
7812,35.0,994.0,203.0,602.0,185.0,3.5865,178000.0
8480,35.0,1281.0,219.0,710.0,184.0,4.8304,152800.0
1051,16.0,1257.0,231.0,559.0,213.0,4.4531,144300.0


### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
scaler = MinMaxScaler(copy=True, feature_range=(0, 1))

In [0]:
X_train[X_train.columns] = scaler.fit_transform(X_train[X_train.columns])
X_test[X_train.columns] = scaler.fit_transform(X_test[X_test.columns])

In [0]:
del X_train['medianHouseValue']
del X_test['medianHouseValue']

In [120]:
X_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
6761,0.352941,0.069688,0.117163,0.048769,0.115442,0.142508
3010,0.607843,0.011242,0.015673,0.008367,0.014142,0.045027
7812,0.666667,0.025230,0.031347,0.020971,0.030258,0.212866
8480,0.666667,0.032530,0.033830,0.024752,0.030094,0.298651
1051,0.294118,0.031919,0.035692,0.019466,0.034863,0.272631


In [121]:
X_test.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
16086,0.686275,0.064243,0.058586,0.020575,0.062094,0.353133
8816,0.705882,0.038015,0.026869,0.010035,0.029369,0.770182
7175,0.901961,0.044846,0.052929,0.033217,0.054122,0.133626
16714,0.313725,0.059996,0.059798,0.026154,0.061045,0.263576
14491,0.411765,0.122929,0.090101,0.034871,0.091882,0.660046


### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [0]:
median_age = tf.feature_column.numeric_column('housingMedianAge')
totalRooms = tf.feature_column.numeric_column('totalRooms')
totalBedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
medianIncome = tf.feature_column.numeric_column('medianIncome')

In [0]:
feat_cols = [median_age, totalRooms, totalBedrooms, population, households, medianIncome]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [0]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=Y_train,
                                                batch_size=10, num_epochs=1000,
                                                shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

> Indented block



In [125]:
model = tf.estimator.DNNRegressor(hidden_units=[6, 6, 6], feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpn_w46c2g', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0f3478d6d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [0]:
model.train(input_fn=input_func, steps=20000)

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [0]:
pred_input_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,
                                                   batch_size=10,
                                                   num_epochs=1,
                                                   shuffle=False)

In [0]:
predictions = model.predict(pred_input_fn)

In [110]:
list(predictions)[:10]

INFO:tensorflow:Could not find trained model in model_dir: /tmp/tmpemse1i7u, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([0.17683673], dtype=float32)},
 {'predictions': array([0.16480483], dtype=float32)},
 {'predictions': array([0.24287882], dtype=float32)},
 {'predictions': array([0.08585201], dtype=float32)},
 {'predictions': array([0.12644613], dtype=float32)},
 {'predictions': array([0.13953182], dtype=float32)},
 {'predictions': array([0.23597406], dtype=float32)},
 {'predictions': array([0.17497797], dtype=float32)},
 {'predictions': array([0.12112237], dtype=float32)},
 {'predictions': array([0.09613479], dtype=float32)}]

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [129]:
final_preds = [pred['predictions'] for pred in predictions]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpn_w46c2g/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [0]:
from sklearn.metrics import mean_squared_error

In [132]:
mean_squared_error(Y_test, final_preds)**0.5

103224.76129279473